In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd

import fiona
import folium

from pathlib import Path

from geowrangler.datasets import geofabrik

import sys

sys.path.append("../../../")  # include parent directory
from src.vector_utils import *

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Get OSM Water features

Water features:
- distance to nearest body of water (man-made or natural)
- check if near the coast or not

### Set-up directories

In [3]:
# data directories
DATA_DIR = Path("../../../data/")
SRC_DIR = Path("../../../src/")
ADMIN_FPATH = DATA_DIR / "01-admin-bounds"
RAW_FPATH = DATA_DIR / "02-raw"
PROCESSED_FPATH = DATA_DIR / "03-processed"
OUTPUT_FPATH = DATA_DIR / "04-output"
GIS_FPATH = DATA_DIR / "05-gis"

### Load administrative boundaries

In [4]:
# Load the administrative boundaries for the 12 cities
admin_bounds_gdf = gpd.read_file(ADMIN_FPATH / "target_admin_bounds.shp")
admin_bounds_gdf.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


### Load coastal buffer

In [5]:
# Load coastal buffer generated from the previous notebook.
coast_buffer = gpd.read_file(PROCESSED_FPATH / "ph_coasts_30m.gpkg")
coast_buffer.head(2)

,uid,geometry
0,0,"POLYGON ((119.20565 4.66970, 119.20750 4.66894..."
1,1,"POLYGON ((119.03071 4.70988, 119.03140 4.70875..."


In [ ]:
osm_water_feats_gdf = add_osm_water_features(
    admin_bounds_gdf,
    "philippines",
    "2022",
)
osm_water_feats_gdf.head(2)

In [7]:
osm_water_feats_df = osm_water_feats_gdf.drop(columns=["geometry"])
osm_water_feats_df = pd.DataFrame(osm_water_feats_df)
osm_water_feats_df.to_csv(OUTPUT_FPATH / "osm_features_water_2022.csv")

In [8]:
osm_waterways_feats_gdf = add_osm_water_features(
    admin_bounds_gdf,
    "philippines",
    "2022",
    waterways=True,
)
osm_waterways_feats_gdf.head(2)

2023-06-07 10:30:22.369 | DEBUG    | src.vector_utils:add_osm_water_features:185 - OSM Water for philippines and year 2022 being loaded from ../../../data/02-raw/osm/philippines-220101-free.shp.zip


,region_nam,region_cod,province_n,province_c,city_name,city_code,barangay_n,barangay_p,geometry,osm_year,osm_river_nearest,osm_stream_nearest,osm_canal_nearest,osm_drain_nearest
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",2022,0.0,2102.032556,1560.588579,2236.652574
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",2022,0.0,105.478349,149.797580,761.117734


In [9]:
osm_waterways_feats_df = osm_waterways_feats_gdf.drop(columns=["geometry"])
osm_waterways_feats_df = pd.DataFrame(osm_waterways_feats_df)
osm_waterways_feats_df.to_csv(OUTPUT_FPATH / "osm_features_waterways_2022.csv")

# Get Distance to Coast

In [8]:
%%time
coast_feat = add_distance_to_shore(
    admin_bounds_gdf, coastal_buffer_path=PROCESSED_FPATH / "ph_coasts_30m.gpkg"
)
coast_feat.head(2)

CPU times: user 9min 54s, sys: 646 ms, total: 9min 55s
Wall time: 9min 55s


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,distance_from_coast
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",308.382889
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",2595.485717


In [9]:
# label coastal barangays
coast_feat.loc[(coast_feat["distance_from_coast"] == 0), "is_coastal"] = True
coast_feat.loc[(coast_feat["distance_from_coast"] != 0), "is_coastal"] = False

In [10]:
coast_feat

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,distance_from_coast,is_coastal
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",308.382889,False
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",2595.485717,False
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",308.382889,False
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((120.34054 16.04489, 120.34054 16.044...",2038.164848,False
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((121.12250 15.58028, 121.12687 15.579...",10000.000000,False
...,...,...,...,...,...,...,...,...,...,...,...
874,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,Namayan,PH137401018,"POLYGON ((121.02328 14.58135, 121.02309 14.581...",4003.212796,False
875,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,Plainview,PH137401022,"POLYGON ((121.03657 14.57933, 121.03734 14.579...",4948.578572,False
876,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,Navotas West,PH137503007,"MULTIPOLYGON (((120.94767 14.65256, 120.94768 ...",0.000000,True
877,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,Tanza,PH137503014,"POLYGON ((120.91841 14.71296, 120.92356 14.708...",0.000000,True


In [11]:
coast_feat_df = coast_feat.drop(
    columns=[
        "ADM1_EN",
        "ADM1_PCODE",
        "ADM2_EN",
        "ADM2_PCODE",
        "ADM3_EN",
        "ADM3_PCODE",
        "ADM4_EN",
        "geometry",
    ]
)
coast_feat_df = pd.DataFrame(coast_feat_df)
coast_feat_df.to_csv(OUTPUT_FPATH / "brgy_dist_to_coast.csv")

In [13]:
# map of poi counts
GOOGLE_BASEMAP = folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Satellite",
    overlay=True,
    control=True,
    show=True,
)

coast_base = coast_buffer.explore(color="white")

coast_feat.explore(column="is_coastal", m=coast_base)

GOOGLE_BASEMAP.add_to(coast_base)

# UNCOMMENT TO DISPLAY MAP
# coast_base

In [14]:
# UNCOMMENT TO SAVE
# coast_base.save(GIS_FPATH / "inspect_coastal_barangays_30m.html")